<a href="https://colab.research.google.com/github/Nganle21/Big-Data-Dreamteam/blob/main/Ngan/PF_songfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

cid = 'e33f0325007b4844bb2d8b79a15f94c1'
secret = 'c4e8f9c0404f4aafa1ef2786d4d2a58d'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
billboard = pd.read_csv('billboardtop.csv')
billboard.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [ ]:
billboard['aSearch'] = billboard['artist'].apply(lambda x : " ".join(x.split(" ")[:2]))
billboard['searcher'] = billboard['song'] + " " + billboard['aSearch']
billboard.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,aSearch,searcher
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,Adele,Easy On Me Adele
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,The Kid,Stay The Kid
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,Lil Nas,Industry Baby Lil Nas
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,Walker Hayes,Fancy Like Walker Hayes
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,Ed Sheeran,Bad Habits Ed Sheeran


In [ ]:
billboard = billboard[billboard["artist"]=="Ed Sheeran"]
billboard = billboard["searcher"].unique()
billboard.shape


(26,)

In [ ]:
billboard = pd.DataFrame(data = billboard, columns = ["searcher"])

In [ ]:
def getTrackURI(searcher):
    result = sp.search(searcher)
    try:
        obj1 = result['tracks']['items'][0]['uri']
        
    except:
        obj1 = None
    return obj1

def getArtists(searcher):
    result = sp.search(searcher)
    try:
        obj2 = result['tracks']['items'][0]['artists']
        names = []
        for name in obj2:
            names.append(name['name'])
        obj2 = ",".join(names)      
    except:
        obj2 = ""
    return obj2



billboard['uri'] = billboard['searcher'].apply(getTrackURI)
billboard['artists'] = billboard['searcher'].apply(getArtists)

billboard = billboard.dropna()
billboard.head()

,searcher,uri,artists
0,Bad Habits Ed Sheeran,spotify:track:3rmo8F54jFF8OgYsqTxm5d,Ed Sheeran
1,Shivers Ed Sheeran,spotify:track:50nfwKoDiSYg8zOCREWAm5,Ed Sheeran
2,Visiting Hours Ed Sheeran,spotify:track:44V6EzbUSYgyfsNiP0FrlA,Ed Sheeran
3,Afterglow Ed Sheeran,spotify:track:0E4Y1XIbs8GrAT1YqVy6dq,Ed Sheeran
4,Perfect Ed Sheeran,spotify:track:0tgVpDi06FyKpA1z0VMD4v,Ed Sheeran


In [ ]:
audio_features = []
for uri in billboard['uri'].values:
    audio_features.append(sp.audio_features(uri)[0])

In [ ]:
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,0.807,0.893,11,-3.745,0,0.0347,0.0451,0.3660,0.537,126.011,230747,4,3.845783
1,0.788,0.859,2,-2.724,1,0.0856,0.2810,0.0424,0.822,141.020,207853,4,3.464217
2,0.471,0.396,8,-6.654,1,0.0336,0.7700,0.0729,0.263,149.609,215507,4,3.591783
3,0.641,0.324,11,-5.851,1,0.0299,0.6980,0.3280,0.273,110.184,185487,4,3.091450
4,0.599,0.448,8,-6.312,1,0.0232,0.1630,0.1060,0.168,95.050,263400,3,4.390000


In [ ]:
billDf = pd.concat([billboard, audio_features], axis=1)

In [ ]:
newOrderURI = "spotify:artist:1uNFoZAHBGtllmzznpCI3s"
results = sp.artist_albums(newOrderURI, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['id'])

    

In [ ]:
a = sp.album_tracks('3uPnO1aZBwMgWK1DI5zve9',limit=50)

In [ ]:
tracks_uri = []
for i in range(len(a)):
    track_uri = a['items'][i]['uri']
    tracks_uri.append(track_uri)
print(tracks_uri)   

['spotify:track:6i4G85fdtDwK6UpQba337E', 'spotify:track:2t96tGXVqmNT9KbbIXyEP1', 'spotify:track:5RE1BlC2bVRL02T0wcLq2X', 'spotify:track:3Ic8V1CEjOY53OVpo2k1Ap', 'spotify:track:2HFHwTr1IBWheGoQtNNKtx', 'spotify:track:7K5ASMnqnCR42bJBJ3GAVz', 'spotify:track:5D0ylGlouZRFfaCLgmm1wA']


In [ ]:
tracks_uri = pd.DataFrame(data = tracks_uri, columns = ['uri'])


In [ ]:
audio_features = []
for uri in tracks_uri.values:
    audio_features.append(sp.audio_features(uri)[0])
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features = pd.concat([tracks_uri,audio_features],axis=1)
audio_features.head()

,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,spotify:track:6i4G85fdtDwK6UpQba337E,0.583,0.444,9,-8.601,1,0.0456,0.5930,0.3530,0.167,119.590,152796,4,2.546600
1,spotify:track:2t96tGXVqmNT9KbbIXyEP1,0.677,0.633,7,-7.831,1,0.0771,0.0314,0.1120,0.409,89.934,187239,4,3.120650
2,spotify:track:5RE1BlC2bVRL02T0wcLq2X,0.595,0.543,11,-8.149,1,0.0380,0.1270,0.0975,0.109,99.928,174406,4,2.906767
3,spotify:track:3Ic8V1CEjOY53OVpo2k1Ap,0.509,0.228,4,-8.680,1,0.0332,0.9200,0.1040,0.586,90.674,156468,4,2.607800
4,spotify:track:2HFHwTr1IBWheGoQtNNKtx,0.670,0.693,6,-8.340,1,0.3500,0.2000,0.0909,0.388,87.017,212093,4,3.534883


In [ ]:
name = 'Justin Bieber'
sp.search(q='artist:' + name, type='artist')

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AJustin+Bieber&type=artist&offset=0&limit=10',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'},
    'followers': {'href': None, 'total': 59797566},
    'genres': ['canadian pop', 'pop'],
    'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s',
    'id': '1uNFoZAHBGtllmzznpCI3s',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5eb8ae7f2aaa9817a704a87ea36',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab676161000051748ae7f2aaa9817a704a87ea36',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f1788ae7f2aaa9817a704a87ea36',
      'width': 160}],
    'name': 'Justin Bieber',
    'popularity': 96,
    'type': 'artist',
    'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/6ZiuwabapK